In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import random
import matplotlib.pyplot as plt

class ConvNet:
    def __init__(self, input_size, hidden_sizes, output_size, kernel_size):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.kernel_size = kernel_size

        # Inicialización de pesos usando Xavier
        self.weights = []
        prev_size = input_size
        for size in hidden_sizes + [output_size]:
            self.weights.append(np.random.randn(prev_size, size) * np.sqrt(2 / prev_size))
            prev_size = size

        self.biases = [np.zeros((1, size)) for size in hidden_sizes + [output_size]]

    def convolutional(self, image):
        image_height, image_width = image.shape
        kernel_height, kernel_width = self.kernel_size, self.kernel_size

        conv_height = image_height - kernel_height + 1
        conv_width = image_width - kernel_width + 1

        conv_output = np.zeros((conv_height, conv_width))

        for i in range(conv_height):
            for j in range(conv_width):
                conv_output[i, j] = np.sum(image[i:i+kernel_height, j:j+kernel_width] * self.conv_weights) + self.conv_bias

        return conv_output

    def maxpooling(self, image, pool_size):
        image_height, image_width = image.shape

        pooled_height = image_height // pool_size
        pooled_width = image_width // pool_size

        pooled_output = np.zeros((pooled_height, pooled_width))

        for i in range(0, image_height, pool_size):
            for j in range(0, image_width, pool_size):
                window = image[i:i+pool_size, j:j+pool_size]
                pooled_output[i//pool_size, j//pool_size] = np.max(window)

        return pooled_output

    def forward(self, image):
        convolution_result = self.convolutional(image)
        pooling_result = self.maxpooling(convolution_result, pool_size=2)
        flattened_result = self.flattening(pooling_result)
        return flattened_result

    def backward(self, image, label, learning_rate=0.001):
        flattened_result = self.forward(image)

        dweights_hidden3_output = np.zeros_like(self.weights_hidden3_output)
        dbias_output = np.zeros_like(self.bias_output)

        scores = np.dot(flattened_result, self.weights_hidden3_output) + self.bias_output
        exp_scores = np.exp(scores)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        probs[0, label] -= 1
        dscores = probs / len(image)

        dweights_hidden3_output += np.dot(flattened_result.reshape(1, -1).T, dscores)
        dbias_output += np.sum(dscores, axis=0, keepdims=True)

        self.weights_hidden3_output -= learning_rate * dweights_hidden3_output
        self.bias_output -= learning_rate * dbias_output

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return np.where(x > 0, 1, 0)

    def xavier_initialization(self, input_size, output_size):
            return np.random.randn(input_size, output_size) * np.sqrt(2 / (input_size + output_size))

    def softmax(self, x):
        exp_scores = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)


train_data = pd.read_csv('/content/drive/MyDrive/oh/fashion-mnist_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/oh/fashion-mnist_test.csv')

X_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values
X_test = test_data.drop('label', axis=1).values
y_test = test_data['label'].values

X_train = X_train / 255.0
X_test = X_test / 255.0
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Ejemplo de visualizacion de 5 imagenes del conjunto train
num_examples_to_show = 5
random_indices = random.sample(range(len(X_train)), num_examples_to_show)

plt.figure(figsize=(10, 5))
for i, idx in enumerate(random_indices):
    plt.subplot(1, num_examples_to_show, i + 1)
    plt.imshow(X_train[idx].reshape(28, 28), cmap='gray')
    plt.title(f"Label: {y_train[idx]}")
    plt.axis('off')

plt.tight_layout()
plt.show()

# Inicializacion de parametros para la creacion de la red
hidden_sizes = [128, 64, 32]
kernel_size = 3
input_size = X_train.shape[1]
hidden_size = 128
output_size = 10

conv_net = ConvNet(input_size, hidden_sizes, output_size, kernel_size)

weights_input_hidden = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / input_size)
bias_hidden = np.zeros(hidden_size)
weights_hidden_output = np.random.randn(hidden_size, output_size) * np.sqrt(2.0 / hidden_size)
bias_output = np.zeros(output_size)

# Momentum para Adam
m_input_hidden = np.zeros_like(weights_input_hidden)
v_input_hidden = np.zeros_like(weights_input_hidden)
m_hidden_output = np.zeros_like(weights_hidden_output)
v_hidden_output = np.zeros_like(weights_hidden_output)

beta1 = 0.9  
beta2 = 0.999  
epsilon = 1e-8 

learning_rate = 0.1
epochs = 300

loss_history = []
accuracy_history = []

for epoch in range(epochs):
    # Feedforward
    hidden_layer_input = np.dot(X_train, weights_input_hidden) + bias_hidden
    hidden_layer_output = np.maximum(0, hidden_layer_input)  # Activacion ReLU
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output

    exp_scores = np.exp(output_layer_input)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    correct_logprobs = -np.log(probs[range(len(X_train)), y_train]) # Entropia cruzada
    data_loss = np.sum(correct_logprobs) / len(X_train)
    loss_history.append(data_loss)

    # Backpropagation
    dscores = probs
    dscores[range(len(X_train)), y_train] -= 1
    dscores /= len(X_train)

    dweights_hidden_output = np.dot(hidden_layer_output.T, dscores)
    dbias_output = np.sum(dscores, axis=0, keepdims=True)
    
    dhidden = np.dot(dscores, weights_hidden_output.T)
    dhidden[hidden_layer_output <= 0] = 0  # Derivada de la funcion ReLU en backpropagation
    dweights_input_hidden = np.dot(X_train.T, dhidden)
    dbias_hidden = np.sum(dhidden, axis=0, keepdims=True)

    m_input_hidden = beta1 * m_input_hidden + (1 - beta1) * dweights_input_hidden
    v_input_hidden = beta2 * v_input_hidden + (1 - beta2) * (dweights_input_hidden ** 2)

    m_hidden_output = beta1 * m_hidden_output + (1 - beta1) * dweights_hidden_output
    v_hidden_output = beta2 * v_hidden_output + (1 - beta2) * (dweights_hidden_output ** 2)

    # Correccion de Adam
    m_input_hidden_corrected = m_input_hidden / (1 - beta1 ** (epoch + 1))
    v_input_hidden_corrected = v_input_hidden / (1 - beta2 ** (epoch + 1))

    m_hidden_output_corrected = m_hidden_output / (1 - beta1 ** (epoch + 1))
    v_hidden_output_corrected = v_hidden_output / (1 - beta2 ** (epoch + 1))

    weights_input_hidden -= learning_rate * m_input_hidden_corrected / (np.sqrt(v_input_hidden_corrected) + epsilon)
    bias_hidden -= learning_rate * dbias_hidden.squeeze()

    weights_hidden_output -= learning_rate * m_hidden_output_corrected / (np.sqrt(v_hidden_output_corrected) + epsilon)
    bias_output -= learning_rate * dbias_output.squeeze()

    hidden_layer = np.maximum(0, np.dot(X_test, weights_input_hidden) + bias_hidden)
    scores = np.dot(hidden_layer, weights_hidden_output) + bias_output
    predicted_class = np.argmax(scores, axis=1)
    test_accuracy = np.mean(predicted_class == y_test)
    accuracy_history.append(test_accuracy)

    print(f"Epoch {epoch+1}/{epochs}. Loss: {data_loss:.4f}. Test Accuracy: {test_accuracy:.4f}")

hidden_layer = np.maximum(0, np.dot(X_test, weights_input_hidden) + bias_hidden)
scores = np.dot(hidden_layer, weights_hidden_output) + bias_output
predicted_class = np.argmax(scores, axis=1)

accuracy = np.mean(predicted_class == y_test)
print(f"\nPrecisión en el conjunto de prueba: {accuracy:.4f}")

# A partir de aqui es tan solo la muestra de matriz confusion, funcion perdida y funcion precision
predictions = predicted_class.ravel()
confusion = confusion_matrix(y_test, predictions)

print("Matriz de Confusión:")
print(confusion)

plt.figure(figsize=(8, 6))
plt.imshow(confusion, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()

tick_marks = range(10) 
plt.xticks(tick_marks, range(10), rotation=45)
plt.yticks(tick_marks, range(10))

for i in range(confusion.shape[0]):
    for j in range(confusion.shape[1]):
        plt.text(j, i, str(confusion[i, j]), ha='center', va='center', color='white')

plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión')

plt.show()

plt.figure(figsize=(8, 6))

plt.subplot(2, 1, 1)
plt.plot(range(len(loss_history)), loss_history, label='Pérdida', color='red')
plt.title('Gráfico de Pérdida')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(range(len(accuracy_history)), accuracy_history, label='Precisión', color='blue')
plt.title('Gráfico de Precisión')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()